In [23]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

# <font color='orange'>01. load data

In [24]:
rootpath = r'C:\Users\w10\Desktop\신용평가모형 세미나\2주차\데이터\temp'

In [25]:
os.listdir(rootpath)

['temp1_inner_total_df.csv',
 'temp2_inner_total_df.csv',
 'temp3_inner_total_df.csv',
 'temp4_inner_total_df.csv',
 'total_df.pkl']

In [26]:
total_df = pd.read_pickle(os.path.join(rootpath, 'total_df.pkl'), )

In [27]:
inner_total_df = pd.read_csv(os.path.join(rootpath, 'temp4_inner_total_df.csv'), )

In [28]:
total_df.head(2)

,no,신청월,직군그룹,근속기간1,웰컴_소득구간,심사원장_소득구간,AS0000136,AS0000137,AS0000138,AS0000139,...,P2O000500_1_s6,LA0000001_1_s9,LA0000020_1_s9,LA0000227_1_s9,P2E000500_1_s9,LA0000001_1_s12,LA0000020_1_s12,LA0000204_1_s12,LA0000227_1_s12,P2O000500_1_s12
0,1,201612,NaN,5.0,5.0,0.0,26,-1,1,0,...,0.0,0.15,0.15,345.45,0.0,0.09,0.09,214.29,214.29,0.0
1,2,201612,D,5.0,2.0,0.0,-1,32,1,0,...,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0


In [29]:
inner_total_df.head(2)

,no,신청월,직군그룹,근속기간1,웰컴_소득구간,심사원장_소득구간,EXECUTE_FG,DESTRUCT_FG,Y_2017_FG,Y_2018_FG,...,ACC_SB_HOUSE_FG,ACC_ML_HOUSE_FG,ACC_P2P_HOUSE_FG,TOTAL_HOUSE_FG,INCOME,INCOME_INTERVAL,UDIR,UDIR_INTERVAL,SEG,FINAL_SEG
0,1,201612,NaN,5.0,5.0,0.0,0,1,1,0,...,0,0,0,0,30000,7,65.92,2,2,1
1,2,201612,D,5.0,2.0,0.0,0,1,1,0,...,0,0,0,0,19000,3,68.25,2,1,1


# <font color='orange'>02. CAP_34 선정

* 직장인 최저생계비(12288) 적중 여부

In [30]:
def cap34(job, income):
    result = 0
    if job!='D' and income<=12288:
        result = 1
        
    return result

In [31]:
input_cols = ['직군그룹','INCOME']

In [32]:
inner_total_df['CAP_34'] = inner_total_df[input_cols].apply(lambda x: cap34(x['직군그룹'], x['INCOME']), axis = 1)

# <font color='orange'>03. 사망자 선정

In [33]:
inner_total_df['THE_DEAD'] = total_df['LIVESTAT2']

# <font color='orange'>04. 개발기간 대상 선정

In [34]:
def not_dev_date(ym):
    result = 0
    if ym>=201811 and ym<=201907:
        result = 1
    elif ym>=201911:
        result = 1
        
    return result

In [35]:
inner_total_df['NOT_DEV_DATE_FG'] = inner_total_df['신청월'].apply(lambda x: not_dev_date(x))

# <font color='red'>05. 최종개발대상자 선정

In [36]:
def final_dev_fg(dup, grade_zero, pre_bad, house, seg, cap, dead, date):
    result = 0
    if dup==0 and grade_zero==0 and pre_bad==0 and house==0 and seg!=0 and cap==0 and dead==0 and date==0:
        result = 1
        
    return result

In [37]:
input_cols = ['FINAL_DUP_FG','GRADE_ZERO_FG','TOTAL_PRE_BAD_FG','TOTAL_HOUSE_FG','FINAL_SEG','CAP_34','THE_DEAD','NOT_DEV_DATE_FG']

In [38]:
inner_total_df['DEV_TARGET_FG'] = inner_total_df[input_cols].apply(lambda x: final_dev_fg(x['FINAL_DUP_FG'],
                                                                                         x['GRADE_ZERO_FG'],
                                                                                         x['TOTAL_PRE_BAD_FG'],
                                                                                         x['TOTAL_HOUSE_FG'],
                                                                                         x['FINAL_SEG'],
                                                                                         x['CAP_34'],
                                                                                         x['THE_DEAD'],
                                                                                         x['NOT_DEV_DATE_FG'],),
                                                                   axis = 1)

In [39]:
inner_total_df['DEV_TARGET_FG'].sum()

190480

# <font color='orange'>06. 저장

In [40]:
inner_total_df.to_csv(os.path.join(rootpath, 'temp5_inner_total_df.csv'), index = False)